**🔒 Proprietary & All Rights Reserved**

**© 2025 Sweety Seelam.** This work is proprietary and protected by copyright. All content, models, code, and visuals are © 2025 Sweety Seelam. 
No part of this project, app, code, or analysis may be copied, reproduced, distributed, or used for any purpose—commercial or otherwise—without explicit written permission from the author.

-------------

# StreamIntel360: A Multi-Agent RAG Platform for Streaming Content & Revenue Intelligence

-----------

# 02 – Embedding Experiments & Similarity Search

This notebook explores:

- How to construct text representations from the Netflix titles catalog.
- How to embed titles using SentenceTransformers.
- How to build a FAISS index.
- How to run example similarity queries (e.g., given a concept or logline).


In [1]:
# Cell 1 - Install dependencies
!pip install sentence-transformers faiss-cpu tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   - -------------------------------------- 10.5/331.9 MB 54.7 MB/s eta 0:00:06
   -- ------------------------------------- 22.0/331.9 MB 53.6 MB/s eta 0:00:06
   --- ------------------------------------ 32.8/331.9 MB 53.4 MB/s eta 0:00:06
   ----- ---------------------------------- 43.5/331.9 MB 52.2 MB/s eta 0:00:06
   ------ --------------------------------- 53.7/331.9 MB 51.1 MB/s eta 0:00:06
   ------- -------------------------------- 64.5/331.9 MB 51.4 MB/s eta 0:00:06
   --------- ------------------------------ 75.2/331.9 MB 51.0 MB/s eta 0:00:06
   ---------- ----------------------------- 87.6/331.9 MB 51.7 MB/s eta 0:00:05
   ------------ --------------------------- 99.6/331.9 MB 52.1 MB/s eta 0:00:05
   ------------- ------------------------- 112.2/331.9 MB 52.7 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.2 which is incompatible.


In [3]:
import tf_keras
tf_keras.__version__

'2.20.1'

In [2]:
# Cell 2 – Code: Imports & Paths
import pandas as pd
import numpy as np
from pathlib import Path

from sentence_transformers import SentenceTransformer
import faiss

DATA_DIR = Path("..") / "data" / "raw"
FILE_PATH = DATA_DIR / "netflix_titles.csv"

FILE_PATH, FILE_PATH.exists()

(WindowsPath('../data/raw/netflix_titles.csv'), True)

In [5]:
# Cell 3 – Code: Load Data & Build Basic Corpus (clean version)

import pandas as pd

cols_to_use = [
    "show_id",
    "type",
    "title",
    "director",
    "cast",
    "country",
    "date_added",
    "release_year",
    "rating",
    "duration",
    "listed_in",
    "description",
]

df = pd.read_csv(
    FILE_PATH,
    encoding="latin-1",
    usecols=cols_to_use,   # <- only keep the real columns
)

df = df.copy()

# Ensure key columns exist
for col in ["title", "description", "listed_in", "type", "country", "release_year"]:
    if col not in df.columns:
        print(f"WARNING: Column {col} not found in dataset")

df["title"] = df["title"].astype(str).str.strip()
df["description"] = df["description"].fillna("").astype(str)
df["listed_in"] = df["listed_in"].fillna("").astype(str)
df["country"] = df["country"].fillna("").astype(str)

df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [6]:
# Cell 4 – Code: Build Text Variants
def build_text_variant(row, variant: str = "baseline") -> str:
    """Build different corpus variants to test which gives better semantic search."""
    if variant == "title_only":
        return row["title"]

    if variant == "title_description":
        return f"Title: {row['title']} | Description: {row['description']}"

    # baseline: include genres and country
    if variant == "baseline":
        parts = [
            f"Title: {row['title']}",
            f"Type: {row.get('type', '')}",
            f"Genres: {row.get('listed_in', '')}",
            f"Country: {row.get('country', '')}",
            f"Year: {row.get('release_year', '')}",
            f"Description: {row['description']}",
        ]
        return " | ".join([p for p in parts if p])

    raise ValueError(f"Unknown variant: {variant}")

VARIANT = "baseline"  # change this to try "title_only" or "title_description"

df["corpus_text"] = df.apply(build_text_variant, axis=1, variant=VARIANT)
df["corpus_text"].head()

0    Title: Dick Johnson Is Dead | Type: Movie | Ge...
1    Title: Blood & Water | Type: TV Show | Genres:...
2    Title: Ganglands | Type: TV Show | Genres: Cri...
3    Title: Jailbirds New Orleans | Type: TV Show |...
4    Title: Kota Factory | Type: TV Show | Genres: ...
Name: corpus_text, dtype: object

In [7]:
# Cell 5 – Code: Load Embedding Model
# A popular all-purpose model; you can choose others later.
model_name = "all-MiniLM-L6-v2"
embed_model = SentenceTransformer(model_name)

print(f"Loaded SentenceTransformer model: {model_name}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\sweet\.anaconda\download\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sweet\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Loaded SentenceTransformer model: all-MiniLM-L6-v2


In [8]:
# Cell 6 – Code: Compute Embeddings
texts = df["corpus_text"].tolist()
len(texts)

# WARNING: This can take some time if dataset is large.
embeddings = embed_model.encode(texts, batch_size=64, show_progress_bar=True)
embeddings.shape

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

(8809, 384)

In [9]:
# Cell 7 – Code: Build FAISS Index
d = embeddings.shape[1]  # embedding dimensionality
index = faiss.IndexFlatL2(d)
index.add(embeddings)

print("Index size:", index.ntotal)

Index size: 8809


In [10]:
# Cell 8 – Helper: Search Function
def search_similar(query: str, k: int = 5):
    """Encode a query and return top-k similar titles with scores."""
    q_emb = embed_model.encode([query])
    distances, indices = index.search(q_emb, k)
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        row = df.iloc[idx]
        results.append(
            {
                "rank": len(results) + 1,
                "title": row["title"],
                "type": row.get("type", ""),
                "distance": float(dist),
                "description": row["description"],
                "genres": row["listed_in"],
                "year": row["release_year"],
            }
        )
    return results

In [11]:
# Cell 9 – Test Query 1
query_1 = "A dark crime thriller about a serial killer in a big city"
results_1 = search_similar(query_1, k=5)
results_1

[{'rank': 1,
  'title': 'Dark Crimes',
  'type': 'Movie',
  'distance': 0.5850995182991028,
  'description': 'A detective on a cold murder case discovers that a famous writerâ\x80\x99s latest novel contains details chillingly similar to the crime heâ\x80\x99s investigating.',
  'genres': 'Dramas, Thrillers',
  'year': 2016},
 {'rank': 2,
  'title': 'Small Town Crime',
  'type': 'Movie',
  'distance': 0.5972224473953247,
  'description': "When a disgraced ex-cop discovers a dying woman, he's compelled to track down her killer â\x80\x94 an act of self-redemption that takes him down a dark path.",
  'genres': 'Thrillers',
  'year': 2017},
 {'rank': 3,
  'title': 'November Criminals',
  'type': 'Movie',
  'distance': 0.7340857982635498,
  'description': "Dissatisfied with the policeâ\x80\x99s progress, a teen and his best friend investigate their classmate's murder, treading down a dark path to the uneasy truth.",
  'genres': 'Dramas, Thrillers',
  'year': 2017},
 {'rank': 4,
  'title': 'N

In [12]:
# Then we can format the results
for r in results_1:
    print(f"#{r['rank']} | {r['title']} ({r['year']}) [{r['type']}]")
    print(f"Genres: {r['genres']}")
    print(f"Distance: {r['distance']:.4f}")
    print(f"Desc: {r['description'][:200]}...")
    print("-" * 80)

#1 | Dark Crimes (2016) [Movie]
Genres: Dramas, Thrillers
Distance: 0.5851
Desc: A detective on a cold murder case discovers that a famous writerâs latest novel contains details chillingly similar to the crime heâs investigating....
--------------------------------------------------------------------------------
#2 | Small Town Crime (2017) [Movie]
Genres: Thrillers
Distance: 0.5972
Desc: When a disgraced ex-cop discovers a dying woman, he's compelled to track down her killer â an act of self-redemption that takes him down a dark path....
--------------------------------------------------------------------------------
#3 | November Criminals (2017) [Movie]
Genres: Dramas, Thrillers
Distance: 0.7341
Desc: Dissatisfied with the policeâs progress, a teen and his best friend investigate their classmate's murder, treading down a dark path to the uneasy truth....
--------------------------------------------------------------------------------
#4 | Night Stalker: The Hunt for a Serial

In [13]:
# Cell 10 – Test Query 2
query_2 = "Feel-good family movie about a dog and children"
results_2 = search_similar(query_2, k=5)

for r in results_2:
    print(f"#{r['rank']} | {r['title']} ({r['year']}) [{r['type']}]")
    print(f"Genres: {r['genres']}")
    print(f"Distance: {r['distance']:.4f}")
    print(f"Desc: {r['description'][:200]}...")
    print("-" * 80)

#1 | Benji (2018) [Movie]
Genres: Children & Family Movies, Dramas
Distance: 0.8095
Desc: A determined dog comes to the rescue and helps heal a broken family when a young boy and his sister stumble into some serious danger....
--------------------------------------------------------------------------------
#2 | Show Dogs (2018) [Movie]
Genres: Children & Family Movies, Comedies
Distance: 0.8127
Desc: A rough and tough police dog must go undercover with an FBI agent as a prim and proper pet at a dog show to save a baby panda from an illegal sale....
--------------------------------------------------------------------------------
#3 | Dog Gone Trouble (2021) [Movie]
Genres: Children & Family Movies, Comedies
Distance: 0.8244
Desc: The privileged life of a pampered dog named Trouble is turned upside-down when he gets lost and must learn to survive on the big-city streets....
--------------------------------------------------------------------------------
#4 | All Dogs Go to Heaven (1989) 

------

## Observations

- The `baseline` text variant (Title + Type + Genres + Country + Year + Description) provides richer context for semantic search.
- Queries like "dark crime thriller" or "family movie about a dog" return reasonably related titles.
- These experiments inform how we design the RAG corpus used by the StreamIntel360 backend:
  - We will likely use a similar combined text structure for embeddings.
  - We know roughly how many neighbors (`k`) give meaningful results (e.g., 5–10).

------

## Summary

***“How should we turn titles into vectors for semantic search?”***

**What I did & why?**

**1.Loaded the same curated columns from netflix_titles.csv.**

- Reason: keep corpus construction consistent with Notebook 1 and the backend.

**2.Built corpus_text with build_text_variant:**

- Variants: "title_only", "title_description", "baseline" (title + type + genres + country + year + description).

- Reason: experiment with different text recipes to see which gives the best semantic behavior.

**3.Loaded SentenceTransformer("all-MiniLM-L6-v2").**

- Reason: a lightweight, free, GPU/CPU-friendly model that works well for semantic search without API keys.

**4.Encoded all 8,809 titles into embeddings.**

- Reason: obtain a dense vector representation for each title that captures semantic meaning.

**5.Built a FAISS index (IndexFlatL2) and added all embeddings.**

- Reason: enable fast nearest-neighbor search over the catalog.

**6.Defined search_similar(query, k) and ran natural queries:**

- “dark crime thriller about a serial killer” → crime thrillers and serial-killer content.

- “feel-good family movie about a dog and children” → Benji, Show Dogs, Dog Gone Trouble, etc.

- Reason: qualitatively validate “does this text recipe + model give intuitive neighbors?”

**What have I achieved?**

- I validated that the baseline corpus design works: rich context (title + type + genres + country + year + description) produces semantically meaningful neighbors.

- I have confirmed FAISS + MiniLM can serve as the RAG retrieval layer for StreamIntel360 on the laptop.

- Now we have a reference implementation of: corpus building → embeddings → FAISS index → similarity search.

---------

## Conclusion


- This notebook therefore establishes the core RAG retrieval engine for StreamIntel360, which the multi-agent backend will use to ground its analysis, find comparable titles, and support executive-style recommendations.

- Transformed that catalog into a searchable semantic space using SentenceTransformers and FAISS.

- Constructed a rich corpus_text per title that blends title, type, genre, country, year, and description into one representation.

- Generated embeddings for all 8,809 titles and built a FAISS index that can return the most similar titles in milliseconds.

- Test queries showed that the system retrieves highly relevant and thematically aligned titles, matching human expectations.

- This notebook proves that the RAG core (retrieval) is sane and effective before wiring it into any backend or UI.

- It answers: “If I describe a concept in natural language, can we surface reasonable titles from the catalog?”

- Yes: the results for both crime thriller and dog-family queries show that this design is good enough to power the AI copilot.

- Practically, it is both a research sandbox and a clean, documented recipe that the backend can mirror.